In [16]:
import numpy as np
import pandas as pd

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

In [19]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [54]:
bs = 100

# MNIST Dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5), std=(0.5))])

train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transform, download=False)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

In [9]:
# def noise():
#     return torch.rand(bs, 100)

In [23]:
class generator(nn.Module):
    def __init__(self, inp, out):
        super(generator,self).__init__()
        
        self.fc1 = nn.Linear(inp, 256)
        self.fc2 = nn.Linear(self.fc1.out_features, self.fc1.out_features*2)
        self.fc3 = nn.Linear(self.fc2.out_features, self.fc2.out_features*2)
        self.fc4 = nn.Linear(self.fc3.out_features, out)
    
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x),0.2)
        x = F.leaky_relu(self.fc2(x),0.2)
        x = F.leaky_relu(self.fc3(x),0.2)
        return torch.tanh(self.fc4(x))

In [63]:
Dis

discriminator(
  (fc1): Linear(in_features=784, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=1, bias=True)
)

In [71]:
class discriminator(nn.Module):
    def __init__(self, inp, out):
        super(discriminator, self).__init__()
        
        self.fc1 = nn.Linear(inp , 1024)
        self.fc2 = nn.Linear(self.fc1.out_features, self.fc1.out_features//2)
        self.fc3 = nn.Linear(self.fc2.out_features, self.fc2.out_features//2)
        self.fc4 = nn.Linear(self.fc3.out_features, out)
    
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x),0.2)
        x = F.dropout(x, 0.3)
        x = F.leaky_relu(self.fc2(x),0.2)
        x = F.dropout(x, 0.3)
        x = F.leaky_relu(self.fc3(x),0.2)
        x = F.dropout(x, 0.3)
        return torch.sigmoid(self.fc4(x))

In [73]:
# train_dataset.train_data.size(1)*train_dataset.train_data.size(2)
# torch.Tensor([0.3])
Dis

discriminator(
  (fc1): Linear(in_features=784, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=1, bias=True)
)

In [82]:
noise_dim = 100
data_dim = train_dataset.train_data.size(1)*train_dataset.train_data.size(2)
Gen = generator(noise_dim, data_dim).to(device)
Dis = discriminator(data_dim, 1).to(device)

In [83]:
lr = 0.0002

#### Generator Optimizer and Loss

criterion_g = nn.BCELoss()
optimizer_g = optim.Adam(Gen.parameters(), lr=lr)

#### Discriminator Optimizer and Loss

criterion_d = nn.BCELoss()
optimizer_d = optim.Adam(Dis.parameters(), lr=lr)

In [84]:
def train_D(x):
    Dis.zero_grad()
    
    data_real, label_real = x.view(-1, data_dim), torch.ones(bs, 1)
    data_real, label_real = Variable(data_real.to(device)), Variable(label_real.to(device))
    
    D_real_output = Dis(data_real)
    D_real_loss = criterion_d(D_real_output, label_real)
    
    
    z_noise = Variable(torch.randn(bs,noise_dim).to(device))
    data_fake, label_fake = Gen(z_noise), Variable(torch.zeros(bs,1).to(device)) 
    
    D_fake_output = Dis(data_fake)
    D_fake_loss = criterion_d(D_fake_output, label_fake)
    
    D_loss = D_real_loss + D_fake_loss
    D_loss.backward()
    optimizer_d.step()
    
    return D_loss.data.item()

In [85]:
def train_G(x):
    Gen.zero_grad()
    
    z_noise = Variable(torch.randn(bs,noise_dim).to(device))
    data_fake, labels_fake = Gen(z_noise), Variable(torch.ones(bs,1).to(device))
    
    D_fake_output = Dis(data_fake)
    G_loss = criterion_g(D_fake_output, labels_fake)
    
    G_loss.backward()
    optimizer_g.step()
    
    return G_loss.data.item()

In [86]:
epochs = 200

for epoch in range(epochs):
    D_losses = [] 
    G_losses = []
    
    for b_idx, (data,_) in enumerate(train_loader):
        D_losses.append(train_D(data))
        G_losses.append(train_G(data))
        
    print('[%d/%d]: D_loss: %.3f, G_loss: %.3f' % (
    (epoch+1), epochs, torch.mean(torch.FloatTensor(D_losses)), torch.mean(torch.FloatTensor(G_losses))))

[1/200]: D_loss: 1.031, G_loss: 2.131
[2/200]: D_loss: 1.081, G_loss: 2.634
[3/200]: D_loss: 0.911, G_loss: 1.762
[4/200]: D_loss: 0.653, G_loss: 2.934
[5/200]: D_loss: 0.489, G_loss: 3.050
[6/200]: D_loss: 0.649, G_loss: 2.602
[7/200]: D_loss: 0.568, G_loss: 2.630
[8/200]: D_loss: 0.589, G_loss: 2.509
[9/200]: D_loss: 0.721, G_loss: 2.149
[10/200]: D_loss: 0.716, G_loss: 2.126
[11/200]: D_loss: 0.659, G_loss: 2.340
[12/200]: D_loss: 0.701, G_loss: 2.295
[13/200]: D_loss: 0.658, G_loss: 2.401
[14/200]: D_loss: 0.725, G_loss: 2.124
[15/200]: D_loss: 0.741, G_loss: 2.106
[16/200]: D_loss: 0.806, G_loss: 1.920
[17/200]: D_loss: 0.799, G_loss: 1.899
[18/200]: D_loss: 0.859, G_loss: 1.742
[19/200]: D_loss: 0.878, G_loss: 1.712
[20/200]: D_loss: 0.908, G_loss: 1.616
[21/200]: D_loss: 0.901, G_loss: 1.631
[22/200]: D_loss: 0.860, G_loss: 1.732
[23/200]: D_loss: 0.879, G_loss: 1.668
[24/200]: D_loss: 0.906, G_loss: 1.614
[25/200]: D_loss: 0.926, G_loss: 1.575
[26/200]: D_loss: 0.968, G_loss: 1

In [89]:
with torch.no_grad():
    test_z = Variable(torch.randn(bs, noise_dim).to(device))
    generated = Gen(test_z)

    save_image(generated.view(generated.size(0), 1, 28, 28), './samples/sample_' + '.png')